<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Business-Intelligence-Anwendungen/blob/main/Decision_Tree_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression Tree, Beispiel Immobilienpreise



## 1 Daten aus Goolge-Drive importieren

In [ ]:
import pandas as pd

In [ ]:
# Google-Drive einbinden
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/DHBW/BI/Daten/"

In [ ]:
# Datentypen für das Laden der Daten definieren
# Beispiel: Zwar handelt es sich beim Baujahr um eine Zahl, allerdings interessiert uns
# hier nur, ob eine Immobilie in einem Jahr gebaut wurde oder eben nicht. 
dtypes = {
    "Verkaufsmonat": "str",
    "Baujahr": "str",
}

In [ ]:
# Relevante Spalten definiere
cols = [
    "Verkaufspreis","Verkaufsmonat","Wohnraum_qm",
    "Keller_qm", "Grundstueck_qm","Baujahr"
    ]

In [ ]:
# Daten laden
data = pd.read_csv(f"{path}house_price.txt", dtype=dtypes, usecols=cols)

In [ ]:
data.dtypes

## 2 Erster Überblick zum Datensatz

In [ ]:
# Kopfzeile der Daten
data.head()

In [ ]:
# Größe des Datensatzes
data.shape

## 3 Prognose mit einem Entscheidungsbaum

In [ ]:
# Features, d.h. womit lässt sich der Immobilienpreis vorhersagen!
X = data.drop(columns="Verkaufspreis")

In [ ]:
# Target, d.h. was soll prognostiziert werden!
y = data["Verkaufspreis"]

In [ ]:
# Kategoriale Features umcodieren
X = pd.get_dummies(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Training Algorithmus: X_train (Features), y_train (Target)
# Test des Algorithmus (Bewertung): X_test, y_test
#     1) Algorithmus bekommt Daten (X_test), die er noch nicht kennt & macht Prognose
#     2) Vergleich Prognose mit tatsächlichen Werte (y_test) --> Bewertung

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
decision_tree = DecisionTreeRegressor(max_depth=3)

In [ ]:
decision_tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
mean_absolute_percentage_error(y_train, decision_tree.predict(X_train)) * 100

In [ ]:
mean_absolute_percentage_error(y_test, decision_tree.predict(X_test)) * 100

## 4 Visualisierung des Entscheidungsbaums

In [ ]:
from sklearn import tree

In [ ]:
# Variante 1
tree.plot_tree(decision_tree)

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
# Variante 2
# 1. Schritt: Exportieren des Entscheidungsbaums in eine .dot Datei
export_graphviz(
    decision_tree,
    out_file="/content/drive/MyDrive/DHBW/BI/tree.dot",
    feature_names=decision_tree.feature_names_in_,
    rounded=True,
    filled=True
)

In [ ]:
# 2. Schritt: Umwandeln der .dot in eine .png Datei
! dot -Tpng /content/drive/MyDrive/DHBW/BI/tree.dot -o /content/drive/MyDrive/DHBW/BI/tree.png

# 5 Bestimmung optimaler Parameter für den Entscheidungsbaum

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Grid definieren
param_grid = {
    "max_depth": [3, 4, 5, 6, 7] # Tiefe des Baums
}

In [ ]:
grid = GridSearchCV(
    decision_tree, # verwendetes Modell
    param_grid, # zu untersuchende Parameter
    cv=10, # Anzahl der Folds
    n_jobs=-1, # Parallelisierung der Ausführung
    scoring="neg_mean_squared_error", # Fehlermaß
)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid_search_results = pd.DataFrame(grid.cv_results_)[
    ["mean_test_score", "std_test_score", "params"]
]
grid_search_results

In [ ]:
# Auswahl des besten Modells
best_model = grid.best_estimator_

In [ ]:
best_model.get_params()

In [ ]:
mean_absolute_percentage_error(y_train, best_model.predict(X_train)) * 100

In [ ]:
mean_absolute_percentage_error(y_test, best_model.predict(X_test)) * 100

In [ ]:
# 1. Schritt: Exportieren des Entscheidungsbaums in eine .dot Datei
export_graphviz(
    best_model,
    out_file="/content/drive/MyDrive/DHBW/BI/best_tree.dot",
    feature_names=decision_tree.feature_names_in_,
    rounded=True,
    filled=True
)

In [ ]:
# 2. Schritt: Umwandeln der .dot in eine .png Datei
! dot -Tpng /content/drive/MyDrive/DHBW/BI/best_tree.dot -o /content/drive/MyDrive/DHBW/BI/best_tree.png

# Random Forest Regressor, Beispiel Immobilienpreise

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor()

In [ ]:
# Grid definieren
param_grid = {
    "max_depth": [6, 7, 9, 12], # maximale Tiefe des Baums
    "n_estimators": [50, 150, 250] # Anzahl der Entscheidungsbäume
}

In [ ]:
grid = GridSearchCV(
    forest, # verwendetes Modell
    param_grid, # zu untersuchende Parameter
    cv=10, # Anzahl der Folds
    n_jobs=-1, # Parallelisierung der Ausführung
    scoring="neg_mean_squared_error", # Fehlermaß
)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
best_model = grid.best_estimator_
best_model.get_params()

In [ ]:
mean_absolute_percentage_error(y_train, best_model.predict(X_train)) * 100

In [ ]:
mean_absolute_percentage_error(y_test, best_model.predict(X_test)) * 100